Para la elaboración de este código me apoyé de la documentación de mesa, el repositorio 'mesa-examples' en Github, y del documento Game_of_Life creado por el profesor.

In [1]:
# Instalación del framework 'mesa'
!pip3 install mesa

     |████████████████████████████████| 668 kB 5.4 MB/s 
     |████████████████████████████████| 60 kB 4.7 MB/s 


In [2]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que puede haber más de un agente por celda elegimos `MultiGrid` que permite más de un objeto por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [173]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
          if content.estado_celda == 1:
            grid[x][y] = 1
          elif content.estado_celda == 2:
            grid[x][y] = 2
          else:
            grid[x][y] = 0
    
    return grid

class Aspiradora(Agent):
    '''
    Representa a una aspiradora que limpia el piso
    '''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.estado_celda = 2
        self.next_position = None

    def step(self):
        neighborhood = self.model.grid.get_neighborhood(
        self.pos,
        moore=True,
        include_center=False)
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        pisos = [obj for obj in this_cell if isinstance(obj, Piso)]

        if pisos[0].estado_celda == 1:
          pisos[0].estado_celda = 0
          self.next_position = self.pos
          self.model.numCeldasLimpias = self.model.numCeldasLimpias + 1
        else:
          self.next_position = self.random.choice(neighborhood)
        
        self.model.movimientos = self.model.movimientos + 1

    def advance(self):
        self.model.grid.move_agent(self, self.next_position)

class Piso(Agent):
    '''
    Representa a un cuadro de piso sucio
    '''
    def __init__(self, unique_id, model, estado_celda):
      super().__init__(unique_id, model)
      self.estado_celda = estado_celda
    
class HabitacionModel(Model):
    '''
    Define el modelo de la habitación
    '''
    def __init__(self, width, height, numAspiradoras, numCeldasSucias):
      self.grid = MultiGrid(width, height, True)
      self.schedule = SimultaneousActivation(self)
      self.numAspiradoras = numAspiradoras
      self.numCeldasSucias = numCeldasSucias
      self.numCeldasLimpias = width * height - numCeldasSucias
      self.movimientos = 0

      sucias = list()
      for i in range(numCeldasSucias):
        sucias.append(self.random.randrange(width*height))

      contador = 0
      for (content, x, y) in self.grid.coord_iter():
        estado = 0

        for i in range(numCeldasSucias):
          if sucias[i] == contador:
            estado = 1

        p = Piso((x, y), self, estado)
        self.grid.place_agent(p, (x, y))
        self.schedule.add(p)
        contador = contador + 1
        

      for i in range (self.numAspiradoras):
        a = Aspiradora(i, self)
        self.grid.place_agent(a, (1, 1))
        self.schedule.add(a)

    
    # Aquí definimos con colector para obtener el grid completo.
      self.datacollector = DataCollector(
      model_reporters={"Grid": get_grid})
    
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()




In [181]:
# Datos de la habitación:
M = int(input("Tamaño horizontal de la habitación: "))
N = int(input("Tamaño vertical de la habitación: "))

# Número de aspiradoras:
NUM_ASPIRADORAS = int(input("Número de aspiradoras: "))

# Porcentaje de celdas inicialmente sucias:
PORCENTAJE_CELDAS_SUCIAS = int(input("Porcentaje de celdas sucias: "))
NUM_CELDAS_SUCIAS = int(M * N * PORCENTAJE_CELDAS_SUCIAS * 0.01)

# Tiempo máximo de ejecución:
TIEMPO_MAXIMO_EJECUCION = float(input("Tiempo máximo de ejecución (segundos): "))

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = HabitacionModel(M, N, NUM_ASPIRADORAS, NUM_CELDAS_SUCIAS)
while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and modelo.numCeldasLimpias < M * N):
  modelo.step()

tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

print("\n\nNúmero de celdas limpias: " + str(modelo.numCeldasLimpias))
print("Número movimientos: " + str(modelo.movimientos))
print("Tiempo de ejecución: " + str(tiempo_ejecucion) + " segundos")

Tamaño horizontal de la habitación: 5
Tamaño vertical de la habitación: 5
Número de aspiradoras: 30
Porcentaje de celdas sucias: 20
Tiempo máximo de ejecución (segundos): 0.8


5Número de celdas limpias: 24
Número movimientos: 75510
Tiempo de ejecución: 0:00:00.800197 segundos


In [175]:
all_grid = modelo.datacollector.get_model_vars_dataframe()

In [176]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [172]:
anim

**REPORTE**

Tras hacer este proyecto me di cuenta de la eficiencia de los agentes cuando los movimientos en la habitación se realizan de manera aleatoria. Concluí que es sumamente ineficiente la randomización ya que las aspiradoras demoran más en limpiar la habitación. Esto lo demuestro en los siguientes casos de prueba. Utilice los mismos parámetros para los 5, cambiando el número de aspiradoras. Ni siquiera con 20 aspiradoras se limpio toda la habitación.



Caso de prueba 1:
Parámetros:
Tamaño horizontal de la habitación: 5
Tamaño vertical de la habitación: 5
Número de aspiradoras: 3
Porcentaje de celdas sucias: 30
Tiempo máximo de ejecución (segundos): 0.8

Salida:
Número de celdas limpias: 23
Número movimientos: 35844
Tiempo de ejecución: 0:00:00.800285 segundos

Caso de prueba 2:
Parámetros:
Tamaño horizontal de la habitación: 5
Tamaño vertical de la habitación: 5
Número de aspiradoras: 10
Porcentaje de celdas sucias: 30
Tiempo máximo de ejecución (segundos): 0.8

Salida:
Número de celdas limpias: 23
Número movimientos: 59840
Tiempo de ejecución: 0:00:00.800144 segundos

Caso de prueba 3:
Parámetros:
Tamaño horizontal de la habitación: 5
Tamaño vertical de la habitación: 5
Número de aspiradoras: 20
Porcentaje de celdas sucias: 30
Tiempo máximo de ejecución (segundos): 0.8

Salida
5Número de celdas limpias: 24
Número movimientos: 71580
Tiempo de ejecución: 0:00:00.800271 segundos

Caso de prueba 4:
Parámetros: 
Tamaño horizontal de la habitación: 5
Tamaño vertical de la habitación: 5
Número de aspiradoras: 15
Porcentaje de celdas sucias: 30
Tiempo máximo de ejecución (segundos): 0.8

Salida:
Número de celdas limpias: 24
Número movimientos: 66975
Tiempo de ejecución: 0:00:00.800234 segundos

Caso de prueba 5:
Parámetros:
Tamaño horizontal de la habitación: 5
Tamaño vertical de la habitación: 5
Número de aspiradoras: 30
Porcentaje de celdas sucias: 20
Tiempo máximo de ejecución (segundos): 0.8

Salida:
Número de celdas limpias: 24
Número movimientos: 75510
Tiempo de ejecución: 0:00:00.800197 segundos